# Importações


In [1]:
import urllib
from itertools import product
from os import getenv
from sqlalchemy import create_engine
from dotenv import load_dotenv

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
from fuzzywuzzy import process

# Connection


connection.py


In [2]:
# cSpell: disable=invalid-name
load_dotenv()


class Connection:
    """
    Class Connection
    """

    def __init__(self):
        """
        Constructor

        Args:
            user (str): user
            password (str): password
            database (str): database
            driver (str): driver
            server (str): server

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        self.__user = getenv("PYMSSQL_USER")
        self.__password = getenv("PYMSSQL_PASSWORD")
        self.__database = getenv("PYMSSQL_DATABASE_AUTOMACAO")
        self.__driver = "{ODBC Driver 17 for SQL Server}"
        self.__server = getenv("PYMSSQL_SERVER")

    def get_connection_automacao(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            # pylint: disable=consider-using-f-string
            conexao_automacao = create_engine(
                "mssql+pyodbc:///?odbc_connect=%s" % params
            )
            return conexao_automacao
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

db_read.py


In [3]:
# cSpell: disable=invalid-name
class Read(Connection):
    """
    Class Read
    Read data from the database and return a pandas dataframe
    Create query to be executed in the database
    """

    # pylint: disable=useless-super-delegation
    def __init__(self):
        """
        Constructor
        """
        super().__init__()

    def get_automacao_data(self, query: str) -> pd.DataFrame:
        """
        Get data from database AUTOMACAO and return a pandas dataframe.

        Parameters
        ----------
        query : str
            Query to be executed in the database

        Returns
        -------
        pandas dataframe
            Dataframe with the query result
        """
        try:
            connection = self.get_connection_automacao()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

    def create_automacao_query(
        self, table: str, where: str = None, orderby: str = None
    ) -> str:
        """
        Create query to be executed in the database AUTOMACAO.

        Parameters
        ----------
        table : str
            Table name
        where : str
            Where clause (optional)
        orderby : str
            Order by clause (optional)

        Returns
        -------
        str
            Query to be executed in the database
        """
        query = f"SELECT * FROM AUTOMACAO.dbo.{table}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

## Criando a query e recebendo dados do Banco de Dados


In [4]:
# cSpell: disable=invalid-name
# Dia de hoje para uso no gráfico
now = pd.to_datetime("today")
today = now.strftime("%Y-%m-%d")

# Encontrando primeiro e último dia do mês anterior
first_day_this_month = now.replace(day=1)  # primeiro dia do mês atual
last_day_month = first_day_this_month - timedelta(
    days=1
)  # último dia do mês anterior
first_day_month = last_day_month.replace(day=1)  # primeiro dia do mês anterior

# mantendo o formato de data e removendo o horário
last_day_month = last_day_month.strftime("%Y-%m-%d")
first_day_month = first_day_month.strftime("%Y-%m-%d")
first_day_this_month = first_day_this_month.strftime("%Y-%m-%d")

print(f"Primeiro dia do mês anterior: {first_day_month}")
print(f"Último dia do mês anterior: {last_day_month}")
print(f"Primeiro dia do mês atual: {first_day_this_month}")

# Instancia a classe Read
DB_read = Read()

# Query para obter os dados do mês anterior
query_occ = DB_read.create_automacao_query(
    table="maquina_ocorrencia",
    where=f"data_registro >= '{first_day_this_month}'",
)

query_info = DB_read.create_automacao_query(
    table="maquina_info", where=f"data_registro >= '{first_day_this_month}'"
)

query_maq_cadastro = DB_read.create_automacao_query(
    table="maquina_cadastro",
    orderby="linha, data_registro DESC, hora_registro DESC",
)

# Leitura do Banco de Dados
df_occ = DB_read.get_automacao_data(query_occ)
df_info = DB_read.get_automacao_data(query_info)
df_maq_cadastro = DB_read.get_automacao_data(query_maq_cadastro)

# df_occ.to_html("teste.html")

Primeiro dia do mês anterior: 2023-12-01
Último dia do mês anterior: 2023-12-31
Primeiro dia do mês atual: 2024-01-01


Testes de Saída do Banco de Dados


In [5]:
df_occ

,recno,maquina_id,motivo_id,problema,solucao,data_registro,hora_registro,usuario_id
0,2638,TMF006,12,,,2024-01-02,01:16:52,000453
1,2639,TMF013,08,,,2024-01-02,01:17:04,000453
2,2640,TMF010,08,,,2024-01-02,01:17:20,000453
3,2641,TMF010,12,,,2024-01-02,01:18:45,000453
4,2642,TMF006,12,,,2024-01-03,08:14:58,000807
...,...,...,...,...,...,...,...,...
220,2858,TMF015,06,Pasta quente,,2024-01-09,13:14:48,000838
221,2859,TMF011,01,Troca do ribbon,,2024-01-09,13:44:38,000838
222,2860,TMF015,07,Falha driver,,2024-01-09,13:50:37,000838
223,2861,TMF004,12,Linha backup,,2024-01-09,15:39:14,000838


In [6]:
df_info

,recno,maquina_id,status,ciclo_1_min,ciclo_15_min,contagem_total_ciclos,contagem_total_produzido,turno,data_registro,hora_registro,tempo_parada,tempo_rodando
0,667577,TMF005,false,0.0,0.0,0.0,0.0,VES,2024-01-01,00:00:14.246666,0.0,0.0
1,667578,TMF002,false,0.0,0.0,0.0,0.0,VES,2024-01-01,00:00:15.253333,0.0,0.0
2,667579,TMF015,false,0.0,0.0,0.0,0.0,VES,2024-01-01,00:00:16.253333,0.0,0.0
3,667580,TMF011,false,0.0,0.0,0.0,0.0,VES,2024-01-01,00:00:17.253333,0.0,0.0
4,667581,TMF014,false,0.0,0.0,0.0,0.0,VES,2024-01-01,00:00:18.253333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
87294,754871,TMF014,true,11.0,150.0,9212.0,9098.0,MAT,2024-01-09,15:50:55.663333,0.0,0.0
87295,754872,TMF004,false,0.0,0.0,220.0,50.0,MAT,2024-01-09,15:50:56.663333,0.0,0.0
87296,754873,TMF009,false,0.0,0.0,626.0,490.0,MAT,2024-01-09,15:50:57.666666,0.0,0.0
87297,754874,TMF001,true,10.0,150.0,9116.0,9032.0,MAT,2024-01-09,15:50:58.666666,0.0,0.0


In [7]:
df_maq_cadastro

,recno,maquina_id,fabrica,linha,data_registro,hora_registro,usuario_id
0,51,TMF003,,0,2023-11-11,12:41:46,000264
1,48,TMF005,,0,2023-11-11,12:37:43,000264
2,47,TMF014,,0,2023-10-29,11:44:53,000264
3,43,TMF005,,0,2023-10-22,12:32:23,000264
4,42,TMF014,,0,2023-10-14,14:58:15,000264
5,39,TMF005,0,0,2023-10-06,15:53:45,000264
6,38,TMF003,,0,2023-10-02,08:17:14,002131
7,37,TMF003,,0,2023-09-30,16:35:54,000264
8,36,TMF003,,0,2023-09-30,16:35:37,000264
9,35,TMF003,,0,2023-09-30,16:34:26,000264


# Service


clean_data.py


In [8]:
class CleanData:
    """
    Classe para limpeza dos dados

    Atributos:
        df (pd.DataFrame): Dataframe com os dados a serem limpos

    Métodos:
        clean_maq_cadastro: Limpa os dados de cadastro das máquinas

    """

    def __init__(self):
        pass

    def clean_maq_cadastro(self, df_cadastro: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados de cadastro das máquinas

        Args:
        -----
            data (pd.DataFrame): Dataframe com os dados a serem limpos

        Retorna:
        --------
            pd.DataFrame: Dataframe com os dados limpos

        Exemplo:
        --------

            >>> from app.service.clean_data import CleanData
            >>> import pandas as pd
            >>> df = pd.DataFrame({'maquina_id': [TMF001, TMF002, TMF003], 'linha': [1, 2, 3],
            'fabrica': [1, 2, 2], 'data_registro': ['2021-01-01', '2021-01-01', '2021-01-01'],
            'hora_registro': ['00:00:00.000', '00:00:00.000', '00:00:00.000'], 'recno': [1, 2, 3],
            usuario_id: [00532, 00533, 00534]})
            >>> clean_data = CleanData()
            >>> df_clean = clean_data.clean_maq_cadastro(df)
            >>> df_clean
                maquina_id  linha  fabrica data_hora_registro       usuario_id
            0     TMF001      1        1    2021-01-01 00:00:00      532
            1     TMF002      2        2    2021-01-01 00:00:00      533
            2     TMF003      3        2    2021-01-01 00:00:00      534


        """

        # Remover rows onde a linha é 0
        df_cadastro = df_cadastro[df_cadastro["linha"] != 0]

        # Remover linhas duplicadas (erros de cadastro)
        df_cadastro = df_cadastro.drop_duplicates(
            subset=["data_registro", "linha"], keep="first"
        )

        # Criar nova coluna combinando data e hora
        df_cadastro["data_hora_registro"] = (
            df_cadastro["data_registro"].astype(str)
            + " "
            + df_cadastro["hora_registro"].astype(str).str.split(".").str[0]
        )

        # Converter coluna data_hora_registro para datetime
        df_cadastro["data_hora_registro"] = pd.to_datetime(
            df_cadastro["data_hora_registro"], format="%Y-%m-%d %H:%M:%S"
        )

        # Remover colunas desnecessárias
        df_cadastro = df_cadastro.drop(
            columns=["recno", "data_registro", "hora_registro"]
        )

        # Ordenar dataframe para facilitar trabalho futuro
        df_cadastro = df_cadastro.sort_values(
            by=["maquina_id", "data_hora_registro"], ascending=[True, False]
        )

        # reiniciar o index
        df_cadastro = df_cadastro.reset_index(drop=True)

        return df_cadastro

    def clean_maq_info(self, df_info: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados de cadastro das máquinas

        Args:
        -----
            data (pd.DataFrame): Dataframe com os dados a serem limpos

        Retorna:
        --------
            pd.DataFrame: Dataframe com os dados limpos

        Exemplo:
        --------

            >>> from app.service.clean_data import CleanData
            >>> import pandas as pd
            >>> df = pd.DataFrame({'maquina_id': [TMF001, TMF002, TMF003], 'status'[False, True, True], 'turno'[MAT, VES, NOT], 'data_registro': ['2021-01-01', '2021-01-01', '2021-01-01'],
            'hora_registro': ['00:00:00.000', '00:00:00.000', '00:00:00.000'], 'recno': [1, 2, 3],
            })
            >>> clean_data = CleanData()
            >>> df_clean = clean_data.clean_maq_info(df)
            >>> df_clean
                maquina_id  status  turno   data_hora_registro      data_hora_final         tempo_registro_min
            0     TMF001    parada    MAT   2021-01-01 00:00:00     2021-01-01 00:01:00     480
            1     TMF002    rodando   VES   2021-01-01 00:00:00     2021-01-01 00:01:00     10
            2     TMF003    rodando   NOT   2021-01-01 00:00:00     2021-01-01 00:01:00     10

        """

        # Ordenar por maquina_id e data_registro, hora_registro
        df_info.sort_values(
            by=["maquina_id", "data_registro", "hora_registro"], inplace=True
        )

        # Criar nova coluna combinando data e hora
        df_info["data_hora_registro"] = (
            df_info["data_registro"].astype(str)
            + " "
            + df_info["hora_registro"].astype(str).str.split(".").str[0]
        )

        # Descartar colunas desnecessárias
        df_info = df_info.drop(
            columns=["recno", "data_registro", "hora_registro"]
        )

        # Corrigindo erros da DB (horário dos turnos) - vai se tornar obsoleto à partir de Fev/24
        df_info.sort_values(
            by=["maquina_id", "data_hora_registro"], inplace=True
        )
        df_info = df_info[
            ~(
                (df_info["turno"] == "VES")
                & (df_info["maquina_id"] != df_info["maquina_id"].shift())
            )
        ]  # descarta a primeira linha se o turno for VES

        # Criar nova coluna status_change para identificar mudança de status
        df_info["status_change"] = df_info["status"].ne(
            df_info["status"].shift()
        )

        # Criar coluna para identificar a mudança de máquina
        df_info["maquina_change"] = df_info["maquina_id"].ne(
            df_info["maquina_id"].shift()
        )

        # criar coluna para consolidar mudança de status e de máquina
        df_info["change"] = (
            df_info["status_change"] | df_info["maquina_change"]
        )

        # Criar coluna para identificar a mudança de turno
        df_info["turno_change"] = df_info["turno"].ne(df_info["turno"].shift())

        # Atualizar coluna change para incluir mudança de turno
        df_info["change"] = df_info["change"] | df_info["turno_change"]

        # Agrupar por maquina e identificar data e hora da última mudança de status
        df_info["change_time"] = (
            df_info.groupby("maquina_id")["data_hora_registro"]
            .shift(0)
            .where(df_info["change"])
        )

        # Remover as linhas onde change_time é nulo
        df_info.dropna(subset=["change_time"], inplace=True)

        # Criar nova coluna com a data_hora_final do status
        df_info["data_hora_final"] = (
            df_info.groupby("maquina_id")["data_hora_registro"]
            .shift(-1)
            .where(~df_info["maquina_change"])
        )

        # Atualizar coluna data_hora_final onde maquina_change é True
        df_info.loc[df_info["maquina_change"], "data_hora_final"] = df_info[
            "change_time"
        ].shift(-1)

        # Remover colunas desnecessárias
        df_info.drop(
            columns=[
                "status_change",
                "maquina_change",
                "turno_change",
                "change",
                "change_time",
            ],
            inplace=True,
        )

        # Remover linhas onde data_hora_final é nulo
        df_info.dropna(subset=["data_hora_final"], inplace=True)

        # Cria nova coluna tempo_registro_min para calcular o tempo de registro em minutos
        df_info["tempo_registro_min"] = (
            pd.to_datetime(df_info["data_hora_final"])
            - pd.to_datetime(df_info["data_hora_registro"])
        ).dt.total_seconds() / 60

        # Arredondar tempo_registro_min e converter para inteiro
        df_info["tempo_registro_min"] = (
            df_info["tempo_registro_min"].round(0).astype(int)
        )

        # Incluir um status in_test para os casos onde o status true permanece por menos de 10 minutos
        df_info = df_info.astype(
            {"status": str}
        )  # para evitar erros de comparação
        df_info.loc[
            (df_info["status"] == "true")
            & (df_info["tempo_registro_min"] <= 10),
            "status",
        ] = "in_test"

        # Ajustar nomenclatura dos status
        df_info.loc[df_info["status"] == "true", "status"] = "rodando"
        df_info.loc[df_info["status"] == "false", "status"] = "parada"

        # Remover colunas desnecessárias
        df_info.drop(
            columns=[
                "ciclo_1_min",
                "ciclo_15_min",
                "contagem_total_ciclos",
                "contagem_total_produzido",
                "tempo_parada",
                "tempo_rodando",
            ],
            inplace=True,
        )

        # Ajustar o index
        df_info.reset_index(drop=True, inplace=True)

        return df_info

    def clean_maq_occ(self, df_occ: pd.DataFrame) -> pd.DataFrame:
        """
        Limpa os dados de cadastro das máquinas

        Args:
        -----
            data (pd.DataFrame): Dataframe com os dados a serem limpos

        Retorna:
        --------
            pd.DataFrame: Dataframe com os dados limpos

        Exemplo:


        """

        # Motivos de Parada
        motivos = {
            1: "Ajustes",
            2: "Troca de Bobina",
            3: "Refeição",
            4: "Reunião",
            5: "Café e Ginástica Laboral",
            6: "Limpeza",
            7: "Manutenção Elétrica",
            8: "Manutenção Mecânica",
            9: "Material em Falta",
            10: "Setup de Sabor",
            11: "Setup de Tamanho",
            12: "Parada Programada",
            13: "Intervenção de Qualidade",
            14: "Linha Cheia",
            15: "Treinamento",
            16: "Limpeza Industrial",
        }

        # Unir as colunas de data e hora
        df_occ["data_hora_registro"] = (
            df_occ["data_registro"].astype(str)
            + " "
            + df_occ["hora_registro"].astype(str).str.split(".").str[0]
        )

        # Modificar coluna motivo_id para int
        df_occ = df_occ.astype({"motivo_id": int})

        # Criar coluna motivo_nome com base no dicionário motivos
        df_occ["motivo_nome"] = df_occ["motivo_id"].map(motivos)

        # Ajustar problema e solucao se a string estiver vazia
        df_occ["problema"] = df_occ["problema"].replace("", np.nan)
        df_occ["solucao"] = df_occ["solucao"].replace("", np.nan)

        # Se o problema for nulo, copiar o motivo_nome para o problema, exceto para os motivos 1, 7, 8, 9, 14,
        df_occ.loc[
            df_occ["problema"].isnull()
            & ~df_occ["motivo_id"].isin([1, 7, 8, 9, 14]),
            "problema",
        ] = df_occ["motivo_nome"]

        # Definir ordem das colunas
        df_occ = df_occ[
            [
                "maquina_id",
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "data_hora_registro",
                "usuario_id",
            ]
        ]

        return df_occ

Saída de Limpeza do Cadastro


In [9]:
clean_data = CleanData()

df_maq_cadastro_clean = clean_data.clean_maq_cadastro(df_maq_cadastro)

df_maq_cadastro_clean

,maquina_id,fabrica,linha,usuario_id,data_hora_registro
0,TMF001,1,8,002131,2023-09-25 10:45:38
1,TMF002,1,2,000264,2023-09-30 16:33:59
2,TMF003,1,5,000264,2023-10-06 15:54:22
3,TMF003,1,2,001996,2023-09-23 15:24:20
4,TMF004,1,6,001996,2023-09-25 11:52:29
5,TMF004,1,6,001996,2023-09-23 15:49:03
6,TMF005,1,1,000264,2023-11-11 12:43:59
7,TMF005,1,1,000264,2023-10-29 11:43:59
8,TMF005,1,9,000264,2023-10-14 14:57:44
9,TMF005,1,5,001996,2023-09-23 15:47:19


Saída da Limpeza de Máquina Info


In [10]:
df_maq_info_clean = clean_data.clean_maq_info(df_info)

df_maq_info_clean

,maquina_id,status,turno,data_hora_registro,data_hora_final,tempo_registro_min
0,TMF001,parada,NOT,2024-01-01 00:02:21,2024-01-01 08:02:22,480
1,TMF001,parada,MAT,2024-01-01 08:02:22,2024-01-01 16:02:24,480
2,TMF001,parada,VES,2024-01-01 16:02:24,2024-01-02 00:02:25,480
3,TMF001,parada,NOT,2024-01-02 00:02:25,2024-01-02 08:02:27,480
4,TMF001,parada,MAT,2024-01-02 08:02:27,2024-01-02 16:02:28,480
...,...,...,...,...,...,...
3081,TMF015,parada,MAT,2024-01-09 15:16:53,2024-01-09 15:18:53,2
3082,TMF015,in_test,MAT,2024-01-09 15:18:53,2024-01-09 15:20:53,2
3083,TMF015,parada,MAT,2024-01-09 15:20:53,2024-01-09 15:22:53,2
3084,TMF015,in_test,MAT,2024-01-09 15:22:53,2024-01-09 15:24:53,2


Saída da Limpeza das Ocorrências


In [11]:
df_maq_occ_clean = clean_data.clean_maq_occ(df_occ)

df_maq_occ_clean

,maquina_id,motivo_id,motivo_nome,problema,solucao,data_hora_registro,usuario_id
0,TMF006,12,Parada Programada,Parada Programada,NaN,2024-01-02 01:16:52,000453
1,TMF013,8,Manutenção Mecânica,NaN,NaN,2024-01-02 01:17:04,000453
2,TMF010,8,Manutenção Mecânica,NaN,NaN,2024-01-02 01:17:20,000453
3,TMF010,12,Parada Programada,Parada Programada,NaN,2024-01-02 01:18:45,000453
4,TMF006,12,Parada Programada,Parada Programada,NaN,2024-01-03 08:14:58,000807
...,...,...,...,...,...,...,...
220,TMF015,6,Limpeza,Pasta quente,NaN,2024-01-09 13:14:48,000838
221,TMF011,1,Ajustes,Troca do ribbon,NaN,2024-01-09 13:44:38,000838
222,TMF015,7,Manutenção Elétrica,Falha driver,NaN,2024-01-09 13:50:37,000838
223,TMF004,12,Parada Programada,Linha backup,NaN,2024-01-09 15:39:14,000838


## Unindo info e ocorrências


join_data.py


In [65]:
class JoinData:
    """
    Classe para unir os dados de cadastro, info e ocorrência
    """

    def __init__(self):
        pass

    def join_info_occ(
        self, df_occ: pd.DataFrame, df_info: pd.DataFrame
    ) -> pd.DataFrame:
        """
        Une os dados de info e ocorrência
        """

        # Garantir que as colunas de data sejam do tipo datetime
        df_occ["data_hora_registro"] = pd.to_datetime(
            df_occ["data_hora_registro"]
        )
        df_info["data_hora_registro"] = pd.to_datetime(
            df_info["data_hora_registro"]
        )
        df_info["data_hora_final"] = pd.to_datetime(df_info["data_hora_final"])

        # Criar uma função para ser usada em cada linha do dataframe
        def merge_rows(row):
            # Selecionar rows onde a data_hora_registro de occ está entre
            # data_hora_registro e data_hora_final de info
            mask = (
                (df_occ["data_hora_registro"] >= row["data_hora_registro"])
                & (df_occ["data_hora_registro"] <= row["data_hora_final"])
                & (df_occ["maquina_id"] == row["maquina_id"])
            )

            # Se houver rows selecionadas, retornar uma serie contendo os valores
            if df_occ.loc[mask].shape[0] > 0:
                return pd.Series(
                    [
                        df_occ.loc[mask, "motivo_id"].values[0],
                        df_occ.loc[mask, "motivo_nome"].values[0],
                        df_occ.loc[mask, "problema"].values[0],
                        df_occ.loc[mask, "solucao"].values[0],
                        df_occ.loc[mask, "data_hora_registro"].values[0],
                        df_occ.loc[mask, "usuario_id"].values[0],
                    ]
                )
            else:
                return pd.Series([None, None, None, None, None, None])

        # Aplicar a função merge_rows em cada linha do dataframe
        df_info[
            [
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "data_hora_registro_operador",
                "usuario_id",
            ]
        ] = df_info.apply(merge_rows, axis=1)

        # Ajustar para sempre que estiver parada por motivo
        # 3, 4, 5, 12, 15, 16, o status será rodando
        df_info.loc[
            (df_info["status"] == "in_test")
            & (df_info["motivo_id"].shift(1).isin([3, 4, 5, 12, 15, 16])),
            "status",
        ] = "rodando"

        # Ajustar o index
        df_info.reset_index(drop=True, inplace=True)

        return df_info

    def adjust_position(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Ajusta a posição das colunas
        """

        # Criar uma coluna parada_inicio como bool para identificar o início de uma parada
        df["parada_inicio"] = df["status"].eq("parada") & df[
            "status"
        ].shift().eq("rodando")

        # Criar uma coluna parada_anterior como bool para identificar a parada anterior
        df["parada_anterior"] = (
            df.groupby("maquina_id")
            .apply(
                lambda group: group.apply(
                    lambda row: row.name if row["parada_inicio"] else np.nan,
                    axis=1,
                )
            )
            .reset_index(level=0, drop=True)
        )

        # Preencher os valores nulos de parada_anterior com o último valor válido
        df["parada_anterior"] = df.groupby("maquina_id")[
            ["parada_anterior"]
        ].ffill()

        # Criar colunas para preparar o movimento das linhas para correção
        df["motivo_id_moved"] = df.groupby("parada_anterior")[
            ["motivo_id"]
        ].transform("first")
        df["motivo_nome_moved"] = df.groupby("parada_anterior")[
            ["motivo_nome"]
        ].transform("first")
        df["problema_moved"] = df.groupby("parada_anterior")[
            ["problema"]
        ].transform("first")
        df["solucao_moved"] = df.groupby("parada_anterior")[
            ["solucao"]
        ].transform("first")
        df["data_hora_registro_operador_moved"] = df.groupby(
            "parada_anterior"
        )[["data_hora_registro_operador"]].transform("first")
        df["usuario_id_moved"] = df.groupby("parada_anterior")[
            ["usuario_id"]
        ].transform("first")

        # Preencher valores nulos com os valores movidos
        df["motivo_id"] = df["motivo_id"].fillna(df["motivo_id_moved"])
        df["motivo_nome"] = df["motivo_nome"].fillna(df["motivo_nome_moved"])
        df["problema"] = df["problema"].fillna(df["problema_moved"])
        df["solucao"] = df["solucao"].fillna(df["solucao_moved"])
        df["data_hora_registro_operador"] = df[
            "data_hora_registro_operador"
        ].fillna(df["data_hora_registro_operador_moved"])
        df["usuario_id"] = df["usuario_id"].fillna(df["usuario_id_moved"])

        # Remover o motivo_id, motivo_nome e problema caso o status seja rodando
        df.loc[
            df["status"] == "rodando",
            [
                "motivo_id",
                "motivo_nome",
                "problema",
                "data_hora_registro_operador",
                "usuario_id",
            ],
        ] = np.nan

        # Preencher os valores nulos com np.nan
        df.fillna(np.nan, inplace=True)

        # Remover as colunas desnecessárias
        df.drop(
            columns=[
                "parada_inicio",
                "parada_anterior",
                "motivo_id_moved",
                "motivo_nome_moved",
                "problema_moved",
                "solucao_moved",
                "data_hora_registro_operador_moved",
                "usuario_id_moved",
            ],
            inplace=True,
        )

        return df

    def info_cadastro_combine(
        self, df_info: pd.DataFrame, df_cadastro: pd.DataFrame
    ) -> pd.DataFrame:
        """
        Une os dados de info e cadastro

        Args:
        -----
            df_info (pd.DataFrame): Dataframe com os dados de info
            df_cadastro (pd.DataFrame): Dataframe com os dados de cadastro

        Retorna:
        --------
            pd.DataFrame: Dataframe com os dados combinados
        """

        # Ordenar os dataframes
        df_info.sort_values(by=["data_hora_registro"], inplace=True)
        df_cadastro.sort_values(by=["data_hora_registro"], inplace=True)

        # Renomear usuario id
        df_cadastro.rename(
            columns={"usuario_id": "usuario_id_maq_cadastro"}, inplace=True
        )
        df_info.rename(
            columns={"usuario_id": "usuario_id_maq_occ"}, inplace=True
        )

        # Merge asof para unir os dataframes baseado na coluna data_hora_registro
        df_info = pd.merge_asof(
            df_info,
            df_cadastro,
            on="data_hora_registro",
            by="maquina_id",
            direction="backward",
        )

        # Reordenar as colunas
        df_info = df_info[
            [
                "maquina_id",
                "linha",
                "fabrica",
                "turno",
                "status",
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "tempo_registro_min",
                "data_hora_registro",
                "data_hora_final",
                "usuario_id_maq_occ",
                "data_hora_registro_operador",
                "usuario_id_maq_cadastro",
                "domingo_feriado_emenda",
            ]
        ]

        # Ordenar pela maquina e hora
        df_info.sort_values(
            by=["maquina_id", "data_hora_registro"],
            ascending=[True, False],
            inplace=True,
        )

        # Ajustar o index
        df_info.reset_index(drop=True, inplace=True)

        return df_info

Saída de join_info_occ


In [66]:
join_data = JoinData()

df_join = join_data.join_info_occ(df_maq_occ_clean, df_maq_info_clean)

df_join

,maquina_id,status,turno,data_hora_registro,data_hora_final,tempo_registro_min,motivo_id,motivo_nome,problema,solucao,data_hora_registro_operador,usuario_id,rodando,grupo
0,TMF001,parada,MAT,2024-01-01 08:02:22,2024-01-01 16:02:24,480,NaN,None,None,NaN,NaT,None,False,1
1,TMF001,parada,MAT,2024-01-02 08:02:27,2024-01-02 16:02:28,480,NaN,None,None,NaN,NaT,None,False,2
2,TMF001,rodando,MAT,2024-01-03 08:02:31,2024-01-03 09:08:31,66,NaN,None,None,NaN,NaT,None,True,3
3,TMF001,parada,MAT,2024-01-03 09:08:31,2024-01-03 09:10:31,2,NaN,None,None,NaN,NaT,None,False,4
4,TMF001,rodando,MAT,2024-01-03 09:10:31,2024-01-03 09:54:31,44,NaN,None,None,NaN,NaT,None,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3081,TMF015,rodando,VES,2024-01-08 23:18:50,2024-01-08 23:48:50,30,NaN,None,None,NaN,NaT,None,True,2156
3082,TMF015,parada,VES,2024-01-08 23:48:50,2024-01-08 23:50:50,2,NaN,None,None,NaN,NaT,None,False,2157
3083,TMF015,in_test,VES,2024-01-08 23:50:50,2024-01-08 23:56:50,6,NaN,None,None,NaN,NaT,None,False,2157
3084,TMF015,parada,VES,2024-01-08 23:56:50,2024-01-09 00:00:50,4,NaN,None,None,NaN,NaT,None,False,2157


Saída após ajustar posição das paradas


In [67]:
df_adjusted = join_data.adjust_position(df_join)

df_adjusted

,maquina_id,status,turno,data_hora_registro,data_hora_final,tempo_registro_min,motivo_id,motivo_nome,problema,solucao,data_hora_registro_operador,usuario_id,rodando,grupo
0,TMF001,parada,MAT,2024-01-01 08:02:22,2024-01-01 16:02:24,480,NaN,NaN,NaN,NaN,NaT,NaN,False,1
1,TMF001,parada,MAT,2024-01-02 08:02:27,2024-01-02 16:02:28,480,NaN,NaN,NaN,NaN,NaT,NaN,False,2
2,TMF001,rodando,MAT,2024-01-03 08:02:31,2024-01-03 09:08:31,66,NaN,NaN,NaN,NaN,NaT,NaN,True,3
3,TMF001,parada,MAT,2024-01-03 09:08:31,2024-01-03 09:10:31,2,NaN,NaN,NaN,NaN,NaT,NaN,False,4
4,TMF001,rodando,MAT,2024-01-03 09:10:31,2024-01-03 09:54:31,44,NaN,NaN,NaN,NaN,NaT,NaN,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3081,TMF015,rodando,VES,2024-01-08 23:18:50,2024-01-08 23:48:50,30,NaN,NaN,NaN,NaN,NaT,NaN,True,2156
3082,TMF015,parada,VES,2024-01-08 23:48:50,2024-01-08 23:50:50,2,NaN,NaN,NaN,NaN,NaT,NaN,False,2157
3083,TMF015,in_test,VES,2024-01-08 23:50:50,2024-01-08 23:56:50,6,NaN,NaN,NaN,NaN,NaT,NaN,False,2157
3084,TMF015,parada,VES,2024-01-08 23:56:50,2024-01-09 00:00:50,4,NaN,NaN,NaN,NaN,NaT,NaN,False,2157


## Deixando apenas dados de parada


get_stops_data.py


In [68]:
class GetStopsData:
    """
    Classe para consolidar os dados de paradas
    """

    def __init__(self):
        pass

    def get_stops_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna um dataframe com os dados de paradas consolidados

        Args:
        -----
            df (pd.DataFrame): Dataframe com os dados de paradas

        Retorna:
        --------
            pd.DataFrame: Dataframe com os dados de paradas consolidados

        Exemplo:
        --------

            >>> from app.service.get_stops_data import GetStopsData
            >>> import pandas as pd
            >>> get_stops_data = GetStopsData()
            >>> df_stops = get_stops_data.get_stops_data(df)

        """

        # Se o motivo_id for nulo, preencher com 0
        df.loc[
            (df["motivo_id"].isnull()) & (df["status"] != "rodando"),
            "motivo_id",
        ] = 0

        # Transformar data e hora em datetime
        df["data_hora_registro_operador"] = pd.to_datetime(
            df["data_hora_registro_operador"]
        )
        df["data_hora_registro"] = pd.to_datetime(df["data_hora_registro"])
        df["data_hora_final"] = pd.to_datetime(df["data_hora_final"])

        # Ordenar por maquina_id, turno, data_hora_registro
        df.sort_values(
            by=["maquina_id", "turno", "data_hora_registro"], inplace=True
        )

        # Criar coluna data_hora_registro_turno para identificar onde está rodando
        df["rodando"] = df["motivo_id"].isnull()

        # Cria uma coluna grupo para identificar os grupos de paradas
        df["grupo"] = (
            (df["rodando"] != df["rodando"].shift())
            | (df["maquina_id"] != df["maquina_id"].shift())
            | (df["turno"] != df["turno"].shift())
            | (
                df["data_hora_registro"].dt.date
                != df["data_hora_registro"].shift().dt.date
            )
        )

        # Soma o tempo de parada por grupo
        df["grupo"] = df["grupo"].cumsum()

        # Agregar os dados por grupo
        df = (
            df.groupby("grupo")
            .agg(
                {
                    "maquina_id": "first",
                    "turno": "first",
                    "status": "first",
                    "motivo_id": "first",
                    "motivo_nome": "first",
                    "problema": "first",
                    "solucao": "first",
                    "tempo_registro_min": "sum",
                    "data_hora_registro": "first",
                    "data_hora_final": "last",
                    "data_hora_registro_operador": "first",
                    "usuario_id": "first",
                }
            )
            .reset_index(drop=True)
        )

        # Ordenar por maquina_id, data_hora_registro
        df.sort_values(by=["maquina_id", "data_hora_registro"], inplace=True)

        # Remover linhas onde a maquina está rodando
        df = df[df["status"] != "rodando"]

        # Substituir status in_test por parada
        df.loc[df["status"] == "in_test", "status"] = "parada"

        # Substituir valores nulos por np.nan
        df.fillna(np.nan, inplace=True)

        # Reiniciar o index
        df.reset_index(drop=True, inplace=True)

        return df

    def dayofweek_adjust(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Ajusta o dia da semana para incluir paradas programadas

        Args:
        -----
            df (pd.DataFrame): Dataframe com os dados de paradas

        Retorna:
        --------
            pd.DataFrame: Dataframe com os dados de paradas ajustados

        Exemplo:
        --------

            >>> from app.service.get_stops_data import GetStopsData
            >>> import pandas as pd
            >>> get_stops_data = GetStopsData()
            >>> df_stops = get_stops_data.dayofweek_adjust(df)

        """

        # Garantir que a coluna data_hora_registro é datetime
        df["data_hora_registro"] = pd.to_datetime(df["data_hora_registro"])

        # Identificar os domingos
        df["domingo"] = df["data_hora_registro"].dt.dayofweek == 6

        # Listar feriados
        feriados = pd.read_csv("../database/feriados.csv")

        # Converter a coluna data para datetime
        feriados["feriados"] = pd.to_datetime(feriados["feriados"])

        # Identificar os feriados
        df["feriado"] = df["data_hora_registro"].dt.date.isin(
            feriados["feriados"].dt.date
        )

        # Identificar os dias após os feriados
        feriados["dia_apos_feriado"] = feriados["feriados"] + pd.Timedelta(
            days=1
        )
        df["dia_apos_feriado"] = df["data_hora_registro"].dt.date.isin(
            feriados["dia_apos_feriado"].dt.date
        )

        # Criar nova coluna unindo domingo, feriado e dia após feriado e descartar
        # as colunas domingo, feriado e dia após feriado
        df["domingo_feriado_emenda"] = (
            df["domingo"] | df["feriado"] | df["dia_apos_feriado"]
        )
        df.drop(
            columns=["domingo", "feriado", "dia_apos_feriado"], inplace=True
        )

        # Definir o status como 12, motivo_nome "Parada Programada" e problema "Domingo/Feriado"
        # para os domingos e feriados onde o motivo_nome é nulo
        df.loc[
            (df["domingo_feriado_emenda"]) & (df["tempo_registro_min"] >= 478),
            ["status", "motivo_id", "motivo_nome", "problema"],
        ] = ["parada", 12, "Parada Programada", "Domingo/Feriado"]

        # Definir como motivo_id 12 e motivo_nome "Parada Programada" se o problema for "Parada Programada"
        df.loc[
            df["problema"] == "Parada Programada",
            ["motivo_id", "motivo_nome"],
        ] = [12, "Parada Programada"]

        return df

    def problems_adjust(self, df: pd.DataFrame, threshold=88) -> pd.DataFrame:
        """
        Ajusta os problemas

        Args:
        -----
            df (pd.DataFrame): Dataframe com os dados de paradas
            threshold (int): Limiar de similaridade para identificar os problemas

        Retorna:
        --------
            pd.DataFrame: Dataframe com os dados de paradas ajustados

        Exemplo:
        --------
            >>> from app.service.get_stops_data import GetStopsData
            >>> import pandas as pd
            >>> get_stops_data = GetStopsData()
            >>> df_stops = get_stops_data.problems_adjust(df)
        """

        # Encontrar problemas únicos
        unique_problems = df["problema"].unique()
        problem_mapping = {}

        # Criar um dicionário para mapear os problemas
        for problem in unique_problems:
            if problem and problem not in problem_mapping:
                problem = str(problem).capitalize()

                # Corrigir erros básicos de digitação
                problem = problem.replace("Beckup", "Backup")
                problem = problem.replace("Becukp", "Backup")
                problem = problem.replace("Stm", "Atm")

                matches = process.extract(
                    problem, unique_problems, limit=len(unique_problems)
                )

                # Encontrar os problemas com maior similaridade
                similar_problems = [
                    match[0] for match in matches if match[1] >= threshold
                ]

                # Criar um dicionário com os problemas similares
                for similar_problem in similar_problems:
                    problem_mapping[similar_problem] = problem

        # Mapear os problemas
        df["problema"] = df["problema"].map(problem_mapping)

        return df

Saída de Get Stops


In [69]:
get_stops = GetStopsData()

df_stops = get_stops.get_stops_data(df_adjusted)

df_stops

,maquina_id,turno,status,motivo_id,motivo_nome,problema,solucao,tempo_registro_min,data_hora_registro,data_hora_final,data_hora_registro_operador,usuario_id
0,TMF001,NOT,parada,0.0,NaN,NaN,NaN,480,2024-01-01 00:02:21,2024-01-01 08:02:22,NaT,NaN
1,TMF001,MAT,parada,0.0,NaN,NaN,NaN,480,2024-01-01 08:02:22,2024-01-01 16:02:24,NaT,NaN
2,TMF001,VES,parada,0.0,NaN,NaN,NaN,480,2024-01-01 16:02:24,2024-01-02 00:02:25,NaT,NaN
3,TMF001,NOT,parada,0.0,NaN,NaN,NaN,480,2024-01-02 00:02:25,2024-01-02 08:02:27,NaT,NaN
4,TMF001,MAT,parada,0.0,NaN,NaN,NaN,480,2024-01-02 08:02:27,2024-01-02 16:02:28,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1163,TMF015,MAT,parada,0.0,NaN,NaN,NaN,30,2024-01-09 09:58:52,2024-01-09 10:28:52,NaT,NaN
1164,TMF015,MAT,parada,3.0,Refeição,Refeição,NaN,148,2024-01-09 10:56:52,2024-01-09 13:24:53,2024-01-09 11:13:08,000838
1165,TMF015,MAT,parada,7.0,Manutenção Elétrica,Falha driver,NaN,22,2024-01-09 13:46:53,2024-01-09 14:08:53,2024-01-09 13:50:37,000838
1166,TMF015,MAT,parada,0.0,NaN,NaN,NaN,2,2024-01-09 14:50:53,2024-01-09 14:52:53,NaT,NaN


In [70]:
df_dayofweek_adjusted = get_stops.dayofweek_adjust(df_stops)

df_dayofweek_adjusted[:25]

,maquina_id,turno,status,motivo_id,motivo_nome,problema,solucao,tempo_registro_min,data_hora_registro,data_hora_final,data_hora_registro_operador,usuario_id,domingo_feriado_emenda
0,TMF001,NOT,parada,12.0,Parada Programada,Domingo/Feriado,NaN,480,2024-01-01 00:02:21,2024-01-01 08:02:22,NaT,NaN,True
1,TMF001,MAT,parada,12.0,Parada Programada,Domingo/Feriado,NaN,480,2024-01-01 08:02:22,2024-01-01 16:02:24,NaT,NaN,True
2,TMF001,VES,parada,12.0,Parada Programada,Domingo/Feriado,NaN,480,2024-01-01 16:02:24,2024-01-02 00:02:25,NaT,NaN,True
3,TMF001,NOT,parada,12.0,Parada Programada,Domingo/Feriado,NaN,480,2024-01-02 00:02:25,2024-01-02 08:02:27,NaT,NaN,True
4,TMF001,MAT,parada,12.0,Parada Programada,Domingo/Feriado,NaN,480,2024-01-02 08:02:27,2024-01-02 16:02:28,NaT,NaN,True
5,TMF001,VES,parada,12.0,Parada Programada,Domingo/Feriado,NaN,478,2024-01-02 16:02:28,2024-01-03 00:00:29,NaT,NaN,True
6,TMF001,VES,parada,0.0,NaN,NaN,NaN,2,2024-01-03 00:00:29,2024-01-03 00:02:29,NaT,NaN,False
7,TMF001,NOT,parada,0.0,NaN,NaN,NaN,24,2024-01-03 00:02:29,2024-01-03 00:26:29,NaT,NaN,False
8,TMF001,NOT,parada,0.0,NaN,NaN,NaN,42,2024-01-03 01:20:30,2024-01-03 02:02:30,NaT,NaN,False
9,TMF001,NOT,parada,0.0,NaN,NaN,NaN,2,2024-01-03 02:38:30,2024-01-03 02:40:30,NaT,NaN,False


Saída após corrigir gramática


In [71]:
problems_adjusted = get_stops.problems_adjust(df_dayofweek_adjusted)

problems_adjusted

,maquina_id,turno,status,motivo_id,motivo_nome,problema,solucao,tempo_registro_min,data_hora_registro,data_hora_final,data_hora_registro_operador,usuario_id,domingo_feriado_emenda
0,TMF001,NOT,parada,12.0,Parada Programada,Domingo/feriado,NaN,480,2024-01-01 00:02:21,2024-01-01 08:02:22,NaT,NaN,True
1,TMF001,MAT,parada,12.0,Parada Programada,Domingo/feriado,NaN,480,2024-01-01 08:02:22,2024-01-01 16:02:24,NaT,NaN,True
2,TMF001,VES,parada,12.0,Parada Programada,Domingo/feriado,NaN,480,2024-01-01 16:02:24,2024-01-02 00:02:25,NaT,NaN,True
3,TMF001,NOT,parada,12.0,Parada Programada,Domingo/feriado,NaN,480,2024-01-02 00:02:25,2024-01-02 08:02:27,NaT,NaN,True
4,TMF001,MAT,parada,12.0,Parada Programada,Domingo/feriado,NaN,480,2024-01-02 08:02:27,2024-01-02 16:02:28,NaT,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,TMF015,MAT,parada,0.0,NaN,Nan,NaN,30,2024-01-09 09:58:52,2024-01-09 10:28:52,NaT,NaN,False
1164,TMF015,MAT,parada,3.0,Refeição,Refeição,NaN,148,2024-01-09 10:56:52,2024-01-09 13:24:53,2024-01-09 11:13:08,000838,False
1165,TMF015,MAT,parada,7.0,Manutenção Elétrica,Falha driver,NaN,22,2024-01-09 13:46:53,2024-01-09 14:08:53,2024-01-09 13:50:37,000838,False
1166,TMF015,MAT,parada,0.0,NaN,Nan,NaN,2,2024-01-09 14:50:53,2024-01-09 14:52:53,NaT,NaN,False


- Foi criado uma função que combina as 3 anteriores: get_stops_data_complete


## Combinando com Maquina Cadastro


Usa join_data.py


In [74]:
df_maq_info_cadastro_combined = join_data.info_cadastro_combine(
    problems_adjusted, df_maq_cadastro_clean
)

df_maq_info_cadastro_combined

,maquina_id,linha,fabrica,turno,status,motivo_id,motivo_nome,problema,solucao,tempo_registro_min,data_hora_registro,data_hora_final,usuario_id_maq_occ,data_hora_registro_operador,usuario_id_maq_cadastro,domingo_feriado_emenda
0,TMF001,8,1,MAT,parada,0.0,NaN,Nan,NaN,16,2024-01-09 14:46:58,2024-01-09 15:02:58,NaN,NaT,002131,False
1,TMF001,8,1,MAT,parada,0.0,NaN,Nan,NaN,2,2024-01-09 14:22:58,2024-01-09 14:24:58,NaN,NaT,002131,False
2,TMF001,8,1,MAT,parada,0.0,NaN,Nan,NaN,2,2024-01-09 13:54:58,2024-01-09 13:56:58,NaN,NaT,002131,False
3,TMF001,8,1,MAT,parada,0.0,NaN,Nan,NaN,2,2024-01-09 13:10:58,2024-01-09 13:12:58,NaN,NaT,002131,False
4,TMF001,8,1,MAT,parada,0.0,NaN,Nan,NaN,8,2024-01-09 12:50:58,2024-01-09 12:58:58,NaN,NaT,002131,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,TMF015,3,1,MAT,parada,12.0,Parada Programada,Domingo/feriado,NaN,480,2024-01-02 08:02:22,2024-01-02 16:02:23,NaN,NaT,001996,True
1164,TMF015,3,1,NOT,parada,12.0,Parada Programada,Domingo/feriado,NaN,480,2024-01-02 00:02:20,2024-01-02 08:02:22,NaN,NaT,001996,True
1165,TMF015,3,1,VES,parada,12.0,Parada Programada,Domingo/feriado,NaN,480,2024-01-01 16:02:19,2024-01-02 00:02:20,NaN,NaT,001996,True
1166,TMF015,3,1,MAT,parada,12.0,Parada Programada,Domingo/feriado,NaN,480,2024-01-01 08:02:17,2024-01-01 16:02:19,NaN,NaT,001996,True
